# 🎾Tennis Match Winner Prediction

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Функция для извлечения признаков из счета матча
def extract_score_features(score):
    if pd.isna(score):
        return [0] * 6
    
    sets = score.split()
    total_sets = len(sets)
    sets_won_1 = 0
    sets_won_2 = 0
    total_games_1 = 0
    total_games_2 = 0
    
    for set_score in sets:
        games = set_score.split('-')
        if len(games) == 2:
            games_1 = int(games[0])
            games_2 = int(games[1])
            total_games_1 += games_1
            total_games_2 += games_2
            if games_1 > games_2:
                sets_won_1 += 1
            else:
                sets_won_2 += 1
    
    return [total_sets, sets_won_1, sets_won_2, 
            total_games_1, total_games_2,
            total_games_1 + total_games_2]

# Функция для создания дополнительных признаков
def create_comparison_features(df):
    features = pd.DataFrame()
    
    # Рейтинговое преимущество (чем больше разница, тем сильнее преимущество)
    features['rank_advantage'] = (df['Rank_2'] - df['Rank_1']) / (df['Rank_1'] + df['Rank_2'])
    
    # Очковое преимущество
    features['points_advantage'] = (df['Pts_1'] - df['Pts_2']) / (df['Pts_1'] + df['Pts_2'])
    
    # Преимущество по коэффициентам
    features['odds_advantage'] = (df['Odd_2'] - df['Odd_1']) / (df['Odd_1'] + df['Odd_2'])
    
    # Бинарные признаки для прямого сравнения
    features['better_rank'] = (df['Rank_1'] < df['Rank_2']).astype(int)
    features['more_points'] = (df['Pts_1'] > df['Pts_2']).astype(int)
    features['better_odds'] = (df['Odd_1'] < df['Odd_2']).astype(int)
    
    # Составной показатель преимущества
    features['overall_advantage'] = (features['better_rank'] + 
                                   features['more_points'] + 
                                   features['better_odds']) / 3
    
    # Добавляем квадраты и взаимодействия признаков
    features['rank_points_interaction'] = features['rank_advantage'] * features['points_advantage']
    features['rank_odds_interaction'] = features['rank_advantage'] * features['odds_advantage']
    features['points_odds_interaction'] = features['points_advantage'] * features['odds_advantage']
    
    return features

# Обновленная функция подготовки данных
def prepare_data(df):
    df = df.copy()
    
    # Конвертируем дату в datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Заменяем отрицательные значения на NaN
    numeric_columns = ['Rank_1', 'Rank_2', 'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2']
    for col in numeric_columns:
        df.loc[df[col] < 0, col] = np.nan
    
    # Извлекаем признаки из счета
    score_features = df['Score'].apply(extract_score_features)
    score_features_df = pd.DataFrame(score_features.tolist(), 
                                   columns=['total_sets', 'sets_won_1', 'sets_won_2',
                                          'total_games_1', 'total_games_2', 'total_games'])
    
    # Создаем признак результата
    df['target'] = (df['Winner'] == df['Player_1']).astype(int)
    
    # Кодируем категориальные признаки
    le = LabelEncoder()
    categorical_columns = ['Series', 'Court', 'Surface', 'Round']
    for col in categorical_columns:
        df[col + '_encoded'] = le.fit_transform(df[col])
    
    # Добавляем новые признаки сравнения
    comparison_features = create_comparison_features(df)
    
    # Объединяем все признаки
    features = pd.concat([
        df[['Rank_1', 'Rank_2', 'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2']],
        df[[col + '_encoded' for col in categorical_columns]],
        score_features_df,
        comparison_features
    ], axis=1)
    
    return features, df['target']

# Функция для обучения модели с настроенными параметрами
def train_model(df_train):
    X, y = prepare_data(df_train)
    
    # Заполняем пропущенные значения
    X = X.fillna(X.mean())
    
    # Создаем модель с увеличенным числом деревьев и настроенными параметрами
    model = RandomForestClassifier(
        n_estimators=200,           # Увеличиваем количество деревьев
        max_depth=10,               # Ограничиваем глубину для избежания переобучения
        min_samples_split=10,       # Минимальное количество образцов для разделения
        min_samples_leaf=5,         # Минимальное количество образцов в листе
        random_state=32,
        class_weight='balanced'     # Балансируем веса классов
    )
    model.fit(X, y)
    
    return model

# Функции get_predictions, create_results_table и display_results остаются без изменений
def get_predictions(model, df_test):
    X_test, _ = prepare_data(df_test)
    X_test = X_test.fillna(X_test.mean())
    probabilities = model.predict_proba(X_test)
    return probabilities

# Функция для создания таблицы результатов
def create_results_table(df_test, predictions):
    results = pd.DataFrame({
        'Player_1': df_test['Player_1'],
        'Player_1_Prediction': predictions[:, 1] * 100,
        'Player_2': df_test['Player_2'],
        'Player_2_Prediction': predictions[:, 0] * 100,
        'Winner': df_test['Winner'],
        'Rank_1': df_test['Rank_1'],
        'Rank_2': df_test['Rank_2'],
        'Pts_1': df_test['Pts_1'],
        'Pts_2': df_test['Pts_2'],
        'Odd_1': df_test['Odd_1'],
        'Odd_2': df_test['Odd_2'],
        'Score': df_test['Score']
    })
    
    # Определяем, кого модель прогнозирует победителем
    results['predicted_winner'] = np.where(predictions[:, 1] > predictions[:, 0], 
                                         df_test['Player_1'], 
                                         df_test['Player_2'])
    
    # Определяем, было ли предсказание верным
    results['correct_prediction'] = (results['predicted_winner'] == df_test['Winner'])
    
    # Добавляем уверенность в прогнозе - максимальная из двух вероятностей
    results['confidence'] = np.maximum(predictions[:, 0], predictions[:, 1]) * 100
    
    return results

# Функция для отображения результатов
def display_results(results):
    styled_results = results.style.format({
        'Player_1_Prediction': '{:.2f}%',
        'Player_2_Prediction': '{:.2f}%',
        'Rank_1': '{:.0f}',
        'Rank_2': '{:.0f}',
        'Pts_1': '{:.0f}',
        'Pts_2': '{:.0f}',
        'Odd_1': '{:.2f}',
        'Odd_2': '{:.2f}'
    }).apply(lambda x: ['background-color: #90EE90' if x['correct_prediction'] 
                       else 'background-color: #FFB6C6' for i in range(len(x))], axis=1)
    
    display(styled_results.hide(axis='columns', subset=['correct_prediction']))

# Функция для оценки и вывода метрик
def display_metrics(model, df_test, predictions, results):
    # Истинные значения (1 - если победил первый игрок, 0 - если второй)
    y_true = (df_test['Winner'] == df_test['Player_1']).astype(int)
    
    # Предсказанные вероятности победы первого игрока
    y_pred_proba = predictions[:, 1]
    
    # Предсказанные бинарные метки
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Расчет метрик
    accuracy = accuracy_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred) * 100
    recall = recall_score(y_true, y_pred) * 100
    f1 = f1_score(y_true, y_pred) * 100
    roc_auc = roc_auc_score(y_true, y_pred_proba) * 100
    entropy_loss = log_loss(y_true, y_pred_proba)
    
    # Матрица ошибок
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    # Выводим метрики
    print(f"\nМетрики качества модели:")
    print(f"Accuracy (Точность): {accuracy:.2f}%")
    print(f"Precision (Точность по положительному классу): {precision:.2f}%")
    print(f"Recall (Полнота): {recall:.2f}%")
    print(f"F1-score (F-мера): {f1:.2f}%")
    print(f"ROC AUC: {roc_auc:.2f}%")
    print(f"Log Loss: {entropy_loss:.4f}")
    
    # Анализ ошибок
    print("\nМатрица ошибок (Confusion Matrix):")
    print(f"True Negative: {tn} (Правильно предсказанные победы второго игрока)")
    print(f"False Positive: {fp} (Неправильно предсказанные победы первого игрока)")
    print(f"False Negative: {fn} (Неправильно предсказанные победы второго игрока)")
    print(f"True Positive: {tp} (Правильно предсказанные победы первого игрока)")
    
    # Расчет метрик по уверенности прогноза
    confidence_bins = [50, 60, 70, 80, 90, 100]
    print("\nТочность предсказаний по уровню уверенности:")
    
    for i in range(len(confidence_bins)-1):
        low = confidence_bins[i]
        high = confidence_bins[i+1]
        
        # Фильтруем предсказания по уровню уверенности
        confident_preds = results[(results['confidence'] >= low) & (results['confidence'] < high)]
        
        if len(confident_preds) > 0:
            bin_accuracy = (confident_preds['correct_prediction'].sum() / len(confident_preds)) * 100
            print(f"Уверенность {low}%-{high}%: {bin_accuracy:.2f}% точность ({len(confident_preds)} матчей)")
        else:
            print(f"Уверенность {low}%-{high}%: нет матчей")
    
    # Выводим важность признаков
    feature_names = (
        ['Rank_1', 'Rank_2', 'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2'] +
        ['Series_encoded', 'Court_encoded', 'Surface_encoded', 'Round_encoded'] +
        ['total_sets', 'sets_won_1', 'sets_won_2', 'total_games_1', 'total_games_2', 'total_games'] +
        ['rank_advantage', 'points_advantage', 'odds_advantage', 'better_rank', 'more_points', 'better_odds', 'overall_advantage', 
         'rank_points_interaction', 'rank_odds_interaction', 'points_odds_interaction']
    )
    
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\nВажность признаков:")
    print(feature_importance.head(10))

## Обучение модели

In [2]:
df_train = pd.read_csv('atp_tennis.csv')

# Обучаем модель
model = train_model(df_train)
print(f"Размер тренировочного набора: {len(df_train)}")

Размер тренировочного набора: 64018


## Тестирование с неполными данными о сетах

In [3]:
df_test = pd.read_csv('atp_tennis_test_minus_one_set.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,De Minaur A.,0.00%,Draper J.,100.00%,Draper J.,10,25,3435,1695,1.67,2.20,3-6 5-7,Draper J.,99.997886
1,Sinner J.,99.90%,Medvedev D.,0.10%,Sinner J.,1,5,9360,6275,1.36,3.20,6-2 1-6 6-1,Sinner J.,99.896842
2,Draper J.,0.00%,Sinner J.,100.00%,Sinner J.,25,1,1695,9360,5.00,1.17,5-7 6-7,Sinner J.,99.999896
3,Fritz T.,51.69%,Tiafoe F.,48.31%,Fritz T.,12,20,3120,2120,1.33,3.40,4-6 7-5 4-6 6-4,Fritz T.,51.692814
4,Fritz T.,0.01%,Sinner J.,99.99%,Sinner J.,12,1,3120,9360,4.00,1.25,3-6 4-6,Sinner J.,99.992263
5,Klein L.,84.80%,Walton A.,15.20%,Klein L.,127,94,468,604,1.73,2.10,7-6,Klein L.,84.802454
6,Sonego L.,33.63%,Daniel T.,66.37%,Daniel T.,50,90,1081,614,1.30,3.50,4-6,Daniel T.,66.369162
7,Mpetshi G.,49.93%,Sun F.,50.07%,Mpetshi G.,51,359,1048,139,1.17,5.00,6-7 7-5,Sun F.,50.072502
8,Coric B.,57.44%,Vukic A.,42.56%,Vukic A.,97,92,582,609,1.91,1.91,6-3 6-7,Coric B.,57.443486
9,Kachmazov A.,87.13%,Kovacevic A.,12.87%,Kachmazov A.,252,72,226,740,2.63,1.50,6-4,Kachmazov A.,87.130384



Метрики качества модели:
Accuracy (Точность): 80.00%
Precision (Точность по положительному классу): 80.00%
Recall (Полнота): 80.00%
F1-score (F-мера): 80.00%
ROC AUC: 92.00%
Log Loss: 0.2922

Матрица ошибок (Confusion Matrix):
True Negative: 4 (Правильно предсказанные победы второго игрока)
False Positive: 1 (Неправильно предсказанные победы первого игрока)
False Negative: 1 (Неправильно предсказанные победы второго игрока)
True Positive: 4 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 33.33% точность (3 матчей)
Уверенность 60%-70%: 100.00% точность (1 матчей)
Уверенность 70%-80%: нет матчей
Уверенность 80%-90%: 100.00% точность (2 матчей)
Уверенность 90%-100%: 100.00% точность (4 матчей)

Важность признаков:
           Feature  Importance
11      sets_won_1    0.350146
12      sets_won_2    0.323941
14   total_games_2    0.111554
13   total_games_1    0.109784
4            Odd_1    0.018413
15     total_games    0.0

## Тестирование с данными об одном сете

In [4]:
df_test = pd.read_csv('atp_tennis_test_one_set.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,De Minaur A.,6.70%,Draper J.,93.30%,Draper J.,10,25,3435,1695,1.67,2.20,3-6,Draper J.,93.299305
1,Sinner J.,94.92%,Medvedev D.,5.08%,Sinner J.,1,5,9360,6275,1.36,3.20,6-2,Sinner J.,94.921121
2,Draper J.,5.67%,Sinner J.,94.33%,Sinner J.,25,1,1695,9360,5.00,1.17,5-7,Sinner J.,94.327752
3,Fritz T.,24.47%,Tiafoe F.,75.53%,Fritz T.,12,20,3120,2120,1.33,3.40,4-6,Tiafoe F.,75.532222
4,Fritz T.,3.53%,Sinner J.,96.47%,Sinner J.,12,1,3120,9360,4.00,1.25,3-6,Sinner J.,96.474916
5,Klein L.,84.80%,Walton A.,15.20%,Klein L.,127,94,468,604,1.73,2.10,7-6,Klein L.,84.802454
6,Sonego L.,33.63%,Daniel T.,66.37%,Daniel T.,50,90,1081,614,1.30,3.50,4-6,Daniel T.,66.369162
7,Mpetshi G.,51.44%,Sun F.,48.56%,Mpetshi G.,51,359,1048,139,1.17,5.00,6-7,Mpetshi G.,51.436383
8,Coric B.,89.81%,Vukic A.,10.19%,Vukic A.,97,92,582,609,1.91,1.91,6-3,Coric B.,89.809284
9,Kachmazov A.,87.13%,Kovacevic A.,12.87%,Kachmazov A.,252,72,226,740,2.63,1.50,6-4,Kachmazov A.,87.130384



Метрики качества модели:
Accuracy (Точность): 80.00%
Precision (Точность по положительному классу): 80.00%
Recall (Полнота): 80.00%
F1-score (F-мера): 80.00%
ROC AUC: 80.00%
Log Loss: 0.5285

Матрица ошибок (Confusion Matrix):
True Negative: 4 (Правильно предсказанные победы второго игрока)
False Positive: 1 (Неправильно предсказанные победы первого игрока)
False Negative: 1 (Неправильно предсказанные победы второго игрока)
True Positive: 4 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 100.00% точность (1 матчей)
Уверенность 60%-70%: 100.00% точность (1 матчей)
Уверенность 70%-80%: 0.00% точность (1 матчей)
Уверенность 80%-90%: 66.67% точность (3 матчей)
Уверенность 90%-100%: 100.00% точность (4 матчей)

Важность признаков:
           Feature  Importance
11      sets_won_1    0.350146
12      sets_won_2    0.323941
14   total_games_2    0.111554
13   total_games_1    0.109784
4            Odd_1    0.018413
15     tot

## Тестирование без данных о сетах

In [5]:
df_test = pd.read_csv('atp_tennis_test_no_score.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,De Minaur A.,43.39%,Draper J.,56.61%,Draper J.,10,25,3435,1695,1.67,2.20,nan,Draper J.,56.613597
1,Sinner J.,74.15%,Medvedev D.,25.85%,Sinner J.,1,5,9360,6275,1.36,3.20,nan,Sinner J.,74.154550
2,Draper J.,14.50%,Sinner J.,85.50%,Sinner J.,25,1,1695,9360,5.00,1.17,nan,Sinner J.,85.504571
3,Fritz T.,66.52%,Tiafoe F.,33.48%,Fritz T.,12,20,3120,2120,1.33,3.40,nan,Fritz T.,66.521986
4,Fritz T.,26.98%,Sinner J.,73.02%,Sinner J.,12,1,3120,9360,4.00,1.25,nan,Sinner J.,73.022827
5,Klein L.,63.56%,Walton A.,36.44%,Klein L.,127,94,468,604,1.73,2.10,nan,Klein L.,63.557647
6,Sonego L.,88.98%,Daniel T.,11.02%,Daniel T.,50,90,1081,614,1.30,3.50,nan,Sonego L.,88.979235
7,Mpetshi G.,88.04%,Sun F.,11.96%,Mpetshi G.,51,359,1048,139,1.17,5.00,nan,Mpetshi G.,88.037774
8,Coric B.,56.46%,Vukic A.,43.54%,Vukic A.,97,92,582,609,1.91,1.91,nan,Coric B.,56.458874
9,Kachmazov A.,41.45%,Kovacevic A.,58.55%,Kachmazov A.,252,72,226,740,2.63,1.50,nan,Kovacevic A.,58.553127



Метрики качества модели:
Accuracy (Точность): 70.00%
Precision (Точность по положительному классу): 66.67%
Recall (Полнота): 80.00%
F1-score (F-мера): 72.73%
ROC AUC: 72.00%
Log Loss: 0.6245

Матрица ошибок (Confusion Matrix):
True Negative: 3 (Правильно предсказанные победы второго игрока)
False Positive: 2 (Неправильно предсказанные победы первого игрока)
False Negative: 1 (Неправильно предсказанные победы второго игрока)
True Positive: 4 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 33.33% точность (3 матчей)
Уверенность 60%-70%: 100.00% точность (2 матчей)
Уверенность 70%-80%: 100.00% точность (2 матчей)
Уверенность 80%-90%: 66.67% точность (3 матчей)
Уверенность 90%-100%: нет матчей

Важность признаков:
           Feature  Importance
11      sets_won_1    0.350146
12      sets_won_2    0.323941
14   total_games_2    0.111554
13   total_games_1    0.109784
4            Odd_1    0.018413
15     total_games    0.01

---

## Тестирование с данными об одном сете (5-ти сетовые матчи)

In [6]:
df_test = pd.read_csv('atp_tennis_test_one_set_47967_big.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,19.19%,Dolgopolov O.,80.81%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7,Dolgopolov O.,80.805388
1,Harrison R.,97.61%,Sela D.,2.39%,Harrison R.,45,95,1115,585,1.30,3.50,6-3,Harrison R.,97.606338
2,Anderson K.,92.06%,Edmund K.,7.94%,Edmund K.,12,49,2610,992,1.40,3.00,7-6,Anderson K.,92.058781
3,Carreno Busta P.,97.93%,Kubler J.,2.07%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5,Carreno Busta P.,97.933222
4,Youzhny M.,18.86%,Cuevas P.,81.14%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7,Cuevas P.,81.139802
5,Istomin D.,90.03%,Herbert P.H.,9.97%,Istomin D.,60,74,848,690,1.66,2.20,6-2,Istomin D.,90.033507
6,Delbonis F.,6.58%,Muller G.,93.42%,Muller G.,67,28,755,1490,4.00,1.25,5-7,Muller G.,93.424199
7,Seppi A.,35.68%,Moutet C.,64.32%,Seppi A.,76,155,686,361,1.33,3.40,3-6,Moutet C.,64.317247
8,Ferrer D.,12.59%,Rublev A.,87.41%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7,Rublev A.,87.409455
9,Ebden M.,81.38%,Isner J.,18.62%,Ebden M.,78,16,684,2265,3.50,1.30,6-4,Ebden M.,81.383962



Метрики качества модели:
Accuracy (Точность): 82.93%
Precision (Точность по положительному классу): 77.78%
Recall (Полнота): 91.80%
F1-score (F-мера): 84.21%
ROC AUC: 90.85%
Log Loss: 0.4323

Матрица ошибок (Confusion Matrix):
True Negative: 46 (Правильно предсказанные победы второго игрока)
False Positive: 16 (Неправильно предсказанные победы первого игрока)
False Negative: 5 (Неправильно предсказанные победы второго игрока)
True Positive: 56 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 25.00% точность (4 матчей)
Уверенность 60%-70%: 84.62% точность (13 матчей)
Уверенность 70%-80%: 50.00% точность (12 матчей)
Уверенность 80%-90%: 89.47% точность (19 матчей)
Уверенность 90%-100%: 89.33% точность (75 матчей)

Важность признаков:
           Feature  Importance
11      sets_won_1    0.350146
12      sets_won_2    0.323941
14   total_games_2    0.111554
13   total_games_1    0.109784
4            Odd_1    0.018413
15   

## Тестирование без данных о сетах (5-ти сетовые матчи)

In [7]:
df_test = pd.read_csv('atp_tennis_test_no_score_47967_big.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,15.81%,Dolgopolov O.,84.19%,Dolgopolov O.,448,37,81,1296,13.00,1.04,nan,Dolgopolov O.,84.194247
1,Harrison R.,91.42%,Sela D.,8.58%,Harrison R.,45,95,1115,585,1.30,3.50,nan,Harrison R.,91.417256
2,Anderson K.,81.04%,Edmund K.,18.96%,Edmund K.,12,49,2610,992,1.40,3.00,nan,Anderson K.,81.035605
3,Carreno Busta P.,86.47%,Kubler J.,13.53%,Carreno Busta P.,11,243,2615,217,1.28,3.75,nan,Carreno Busta P.,86.472937
4,Youzhny M.,55.96%,Cuevas P.,44.04%,Cuevas P.,90,34,604,1345,2.10,1.72,nan,Youzhny M.,55.955936
5,Istomin D.,51.65%,Herbert P.H.,48.35%,Istomin D.,60,74,848,690,1.66,2.20,nan,Istomin D.,51.646158
6,Delbonis F.,39.27%,Muller G.,60.73%,Muller G.,67,28,755,1490,4.00,1.25,nan,Muller G.,60.730873
7,Seppi A.,90.72%,Moutet C.,9.28%,Seppi A.,76,155,686,361,1.33,3.40,nan,Seppi A.,90.717232
8,Ferrer D.,48.58%,Rublev A.,51.42%,Rublev A.,33,32,1360,1373,1.90,1.90,nan,Rublev A.,51.419439
9,Ebden M.,35.35%,Isner J.,64.65%,Ebden M.,78,16,684,2265,3.50,1.30,nan,Isner J.,64.652994



Метрики качества модели:
Accuracy (Точность): 73.98%
Precision (Точность по положительному классу): 67.90%
Recall (Полнота): 90.16%
F1-score (F-мера): 77.46%
ROC AUC: 75.15%
Log Loss: 0.6166

Матрица ошибок (Confusion Matrix):
True Negative: 36 (Правильно предсказанные победы второго игрока)
False Positive: 26 (Неправильно предсказанные победы первого игрока)
False Negative: 6 (Неправильно предсказанные победы второго игрока)
True Positive: 55 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 72.73% точность (22 матчей)
Уверенность 60%-70%: 60.00% точность (25 матчей)
Уверенность 70%-80%: 85.19% точность (27 матчей)
Уверенность 80%-90%: 78.38% точность (37 матчей)
Уверенность 90%-100%: 66.67% точность (12 матчей)

Важность признаков:
           Feature  Importance
11      sets_won_1    0.350146
12      sets_won_2    0.323941
14   total_games_2    0.111554
13   total_games_1    0.109784
4            Odd_1    0.018413
15  

---

## Тестирование с данными об одном сете (высокорейтинговые теннисисты)

In [8]:
df_test = pd.read_csv('atp_tennis_test_new_one_set.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Fritz T.,2.88%,Sinner J.,97.12%,Sinner J.,5,1,4300,10330,5.50,1.14,4-6,Sinner J.,97.121746
1,Zverev A.,20.79%,Fritz T.,79.21%,Fritz T.,2,5,7315,4300,1.40,3.00,3-6,Fritz T.,79.207580
2,Sinner J.,93.22%,Ruud C.,6.78%,Sinner J.,1,7,10330,3855,1.05,11.00,6-1,Sinner J.,93.215702
3,Alcaraz C.,23.60%,Zverev A.,76.40%,Zverev A.,3,2,6810,7315,1.91,1.91,6-7,Zverev A.,76.397419
4,Ruud C.,84.95%,Rublev A.,15.05%,Ruud C.,7,8,3855,3760,2.50,1.53,6-4,Ruud C.,84.948201
5,Fritz T.,91.40%,De Minaur A.,8.60%,De Minaur A.,5,9,4300,3745,1.36,3.20,7-5,Fritz T.,91.402640
6,Sinner J.,94.00%,Medvedev D.,6.00%,Sinner J.,1,4,10330,4830,1.17,5.00,6-3,Sinner J.,94.000096
7,Rublev A.,7.27%,Alcaraz C.,92.73%,Alcaraz C.,8,3,3760,6810,2.50,1.53,3-6,Alcaraz C.,92.726971
8,Zverev A.,90.52%,Ruud C.,9.48%,Zverev A.,2,7,7315,3855,1.13,6.00,7-6,Zverev A.,90.516465
9,De Minaur A.,11.55%,Medvedev D.,88.45%,Medvedev D.,9,4,3745,4830,1.91,1.91,2-6,Medvedev D.,88.447995



Метрики качества модели:
Accuracy (Точность): 92.86%
Precision (Точность по положительному классу): 85.71%
Recall (Полнота): 100.00%
F1-score (F-мера): 92.31%
ROC AUC: 95.83%
Log Loss: 0.2949

Матрица ошибок (Confusion Matrix):
True Negative: 7 (Правильно предсказанные победы второго игрока)
False Positive: 1 (Неправильно предсказанные победы первого игрока)
False Negative: 0 (Неправильно предсказанные победы второго игрока)
True Positive: 6 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: нет матчей
Уверенность 60%-70%: нет матчей
Уверенность 70%-80%: 100.00% точность (3 матчей)
Уверенность 80%-90%: 100.00% точность (3 матчей)
Уверенность 90%-100%: 87.50% точность (8 матчей)

Важность признаков:
           Feature  Importance
11      sets_won_1    0.350146
12      sets_won_2    0.323941
14   total_games_2    0.111554
13   total_games_1    0.109784
4            Odd_1    0.018413
15     total_games    0.017441
5         

## Тестирование без данных о сетах (высокорейтинговые теннисисты)

In [9]:
df_test = pd.read_csv('atp_tennis_test_new_no_score.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Fritz T.,16.89%,Sinner J.,83.11%,Sinner J.,5,1,4300,10330,5.50,1.14,nan,Sinner J.,83.113290
1,Zverev A.,70.64%,Fritz T.,29.36%,Fritz T.,2,5,7315,4300,1.40,3.00,nan,Zverev A.,70.638577
2,Sinner J.,70.44%,Ruud C.,29.56%,Sinner J.,1,7,10330,3855,1.05,11.00,nan,Sinner J.,70.440619
3,Alcaraz C.,50.77%,Zverev A.,49.23%,Zverev A.,3,2,6810,7315,1.91,1.91,nan,Alcaraz C.,50.767058
4,Ruud C.,36.24%,Rublev A.,63.76%,Ruud C.,7,8,3855,3760,2.50,1.53,nan,Rublev A.,63.763323
5,Fritz T.,67.04%,De Minaur A.,32.96%,De Minaur A.,5,9,4300,3745,1.36,3.20,nan,Fritz T.,67.044964
6,Sinner J.,74.64%,Medvedev D.,25.36%,Sinner J.,1,4,10330,4830,1.17,5.00,nan,Sinner J.,74.642514
7,Rublev A.,58.83%,Alcaraz C.,41.17%,Alcaraz C.,8,3,3760,6810,2.50,1.53,nan,Rublev A.,58.825765
8,Zverev A.,73.26%,Ruud C.,26.74%,Zverev A.,2,7,7315,3855,1.13,6.00,nan,Zverev A.,73.263893
9,De Minaur A.,54.00%,Medvedev D.,46.00%,Medvedev D.,9,4,3745,4830,1.91,1.91,nan,De Minaur A.,53.995192



Метрики качества модели:
Accuracy (Точность): 50.00%
Precision (Точность по положительному классу): 45.45%
Recall (Полнота): 83.33%
F1-score (F-мера): 58.82%
ROC AUC: 79.17%
Log Loss: 0.6659

Матрица ошибок (Confusion Matrix):
True Negative: 2 (Правильно предсказанные победы второго игрока)
False Positive: 6 (Неправильно предсказанные победы первого игрока)
False Negative: 1 (Неправильно предсказанные победы второго игрока)
True Positive: 5 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 25.00% точность (4 матчей)
Уверенность 60%-70%: 0.00% точность (2 матчей)
Уверенность 70%-80%: 71.43% точность (7 матчей)
Уверенность 80%-90%: 100.00% точность (1 матчей)
Уверенность 90%-100%: нет матчей

Важность признаков:
           Feature  Importance
11      sets_won_1    0.350146
12      sets_won_2    0.323941
14   total_games_2    0.111554
13   total_games_1    0.109784
4            Odd_1    0.018413
15     total_games    0.0174